<a href="https://colab.research.google.com/github/pinholuc/mlops/blob/master/ml_handbook/concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import ensemble, preprocessing, tree
from sklearn.metrics import auc, confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split, StratifiedKFold

from yellowbrick.classifier import ConfusionMatrix, ROCAUC
from yellowbrick.model_selection import LearningCurve

# Read titanic survival data
df = pd.read_excel("./titanic/titanic3.xls")

# Droping features that are not useful to our model
df = df.drop(columns = ["name", 
                        "ticket", 
                        "home.dest", 
                        "boat", 
                        "body", 
                        "cabin"])

# Getting rid of string/object features
df = pd.get_dummies(df).drop(columns="sex_male")

# Segregate features (x) and labels (y)
X = df.drop(columns="survived")

# Labels (y)
y = df.survived

# Split in train and validation
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=42)



In [ ]:
# Filling Missing Data

df.age.value_counts(dropna=False)

from sklearn.experimental import enable_iterative_imputer
from sklearn import impute

num_cols = ["pclass", "age", "sibsp", "parch", "fare", "sex_female"]
imputer = impute.IterativeImputer()
imputed = imputer.fit_transform(X_train[num_cols])
X_train.loc[:, num_cols] = imputed
imputed = imputer.transform(X_test[num_cols])
X_test.loc[:, num_cols] = imputed

In [ ]:
# Normalizing data

"""

media igual a zero e desvio padrao igual a 1
desse modo, os modelos nao tratarao as variaveis com escalas maiores como
mais importante que as variaveis com menor escala

"""

cols = "pclass, age, sibsp, fare".split(",")
sca = preprocessing.StandardScaler()

X_train = sca.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns=cols)
X_test = sca.transform(X_test)
X_test = pd.DataFrame(X_test, columns=cols)


In [ ]:
# Utilizando modelos de base para benchmark

from sklearn.dummy import DummyClassifier
bm = DummyClassifier()
bm.fit(X_train, y_train)
bm.score(X_test, y_test)



0.5699745547073791

In [ ]:
# K-fold validacao cruzada

"""
KFOLD: https://drigols.medium.com/introdu%C3%A7%C3%A3o-a-valida%C3%A7%C3%A3o-cruzada-k-fold-2a6bced32a90
Outra observação muito importante é que a Validação-Cruzada: K-Fold não 
retorma um modelo (Por exemplo, Regressão Linear) pronto para nós utilizarmos.
 Ele retorna os scores de cada subdivisão, ou seja, 
quão performático cada uma é.

AUC: area under ROC curve. ROC = Receiver operating characteristic
https://en.wikipedia.org/wiki/Receiver_operating_characteristic

The ROC curve is created by plotting the true positive rate (TPR) against the 
false positive rate (FPR) at various threshold settings.
The true-positive rate is also known as sensitivity, recall or
probability of detection

"""

X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

from sklearn import model_selection
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier
import xgboost


for model in [
    DummyClassifier,
    DecisionTreeClassifier,
    KNeighborsClassifier,
    GaussianNB,
    SVC,
    RandomForestClassifier,
    xgboost.XGBClassifier
]:

  cls = model()
  kfold = model_selection.KFold(n_splits=10, random_state=42, shuffle=True)
  s = model_selection.cross_val_score(cls, X, y, scoring="roc_auc", cv=kfold)

  print(f"{model.__name__} AUC: {s.mean()}, STD: {s.std()}")

DummyClassifier AUC: 0.5, STD: 0.0
DecisionTreeClassifier AUC: 0.7547693750229326, STD: 0.02317072542938455
KNeighborsClassifier AUC: 0.7187329287747022, STD: 0.040277951013247305
GaussianNB AUC: 0.8042960782658678, STD: 0.04438480209202408
SVC AUC: 0.7412092038442221, STD: 0.057045949345137584
RandomForestClassifier AUC: 0.8474062722238699, STD: 0.02103449935511667
XGBClassifier AUC: 0.862312412857627, STD: 0.026543685197120637


In [ ]:
# Criando um modelo RandomForest

"""

Uma boa vantagem dos modelos baseados em arvore eh que voce pode inspecionar
a importancia dos atributos. A importancia dos atributos nos diz quanto
um atributo contribui para o modelo. Observe que remover um atributo nao
significa que a pontuacao caira na mesma medida, pois outros atributos 
poderao ser colineares.

A importancia dos atributos eh calculada observando o aumento no erro.
Se a remocao de um atributo causar um aumento no erro do modelo, eh sinal
de que o atributo eh importante.

"""

rf = ensemble.RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)
metrics.precision_score(y_test, rf.predict(X_test))

for col, val in sorted(zip(X_train.columns, rf.feature_importances_),
                       key=lambda x: x[1],
                       reverse=True)[:5]:
  print(f"{col:10}{val:10.3f}")





age            0.281
fare           0.263
sex_female     0.241
pclass         0.090
sibsp          0.049


In [ ]:
# Otimizando hiperparametros

"""
O sklearn tem uma classe de busca em grade (grid search) para avaliar
um modelo com diferentes combinacoes de parametros, devolvendo o melhor
resultado
""" 

rf4 = ensemble.RandomForestClassifier()
params = {
    "max_features": [0.4, "auto"],
    "n_estimators": [15, 200],
    "min_samples_leaf": [1, 0.1],
    "random_state": [42]
}

cv = model_selection.GridSearchCV(rf4, params, n_jobs=1).fit(X_train, y_train)
print(cv.best_params_)

{'max_features': 0.4, 'min_samples_leaf': 1, 'n_estimators': 200, 'random_state': 42}


In [ ]:
# Confusion Matrix

"""
Nos permite ver as classificacoes corretas, bem como os falso-positivos e
os falsos negativos
"""
from sklearn.metrics import confusion_matrix
y_pred = rf.predict(X_test)
confusion_matrix(y_test, y_pred)

mapping = {0: "died", 1: "survived"}
fig, ax = plt.subplots(figsize=(6,6))
cm_viz = ConfusionMatrix(rf, classes=["died", "survived"], label_encoder=mapping)
cm_viz.score(X_test, y_test)
cm_viz.poof()
fig.savefig("mlpr_0304.png", dpi=300, bbox_inches="tight")



/usr/local/lib/python3.8/dist-packages/yellowbrick/classifier/base.py:232: YellowbrickWarning: could not determine class_counts_ from previously fitted classifier
  warnings.warn(


In [ ]:
# Curva ROC

"""

Um grafico ROC (Receiver Operating Characteristic) eh uma ferramenta comum usada
para avaliar classificadores. Ao calcular a area sob a curva ROC (AUC), podemos 
obter uma metrica para compara diferentes classificadores.

Mostra a taxa dos realmente positivos em relacao a taxa dos falso-positivos.
Em geral, quanto mais saliente, melhor. Calcular o AUC fornece um unico numero
a ser avaliado. Um valor mais proximo de um eh melhor. Abaixo de 0,5 o modelo
eh considerado ruim

"""

y_pred = rf.predict(X_test)
roc_auc_score(y_test, y_pred)

fig, ax = plt.subplots(figsize=(6,6))
roc_viz = ROCAUC(rf)
roc_viz.poof()
fig.savefig("roc.png")

/usr/local/lib/python3.8/dist-packages/yellowbrick/base.py:236: YellowbrickWarning: ROCAUC does not have a reference to a matplotlib.Axes the figure may not render as expected!
  warnings.warn(


In [ ]:
# Learning Curve

"""

Uma curva de aprendizado eh usada para nos dizer se temos dados de treinamento
suficientes. O modelo eh treinado com porcoes cada vez maiores dos dados e a
pontuacao eh calculada.

Se a pontuacao da validacao cruzada continuar subindo, talvez seka
necessario investir em coletas de mais dados.

"""
import numpy as np

fig, ax = plt.subplots(figsize=(6,6))
cv = StratifiedKFold(12)
sizes = np.linspace(0.3, 1.0, 10)
lc_viz = LearningCurve(rf,
                       cv=cv,
                       train_sizes=sizes,
                       scoring="f1_weighted",
                       n_jobs=4,
                       ax=ax)
lc_viz.fit(X, y)
lc_viz.poof()
fig.savefig("learning_curve.png")



In [ ]:
# Implantando um modelo

"""
Ao usar o modulo pickle do Python, podemos fazer a persistencia dos modelos
e carrega-los. Depois que tivermos um modelo, chamamos o metodo .predict para
obter uma classificacao ou um resultado de uma regressao.

"""

import pickle
pic = pickle.dumps(rf)
model = pickle.loads(pic)
y_pred = model.predict(X_test)
roc_auc_score(y_test, y_pred)

0.7814217032967034